In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Admin\\Desktop\\code\\python\\Mlops-Production-Ready-Deep-Learning-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Admin\\Desktop\\code\\python\\Mlops-Production-Ready-Deep-Learning-Project'

### Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories


In [7]:
class ConfigurationManager:
    def __init__(self,config_file_path = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH):
        self.Config = read_yaml(config_file_path)
        self.Params = read_yaml(params_file_path)
         
        create_directories([self.Config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.Config.data_ingestion

        create_directories([config.root_dir])
        

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config


        
        
   

In [8]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        


    def download_file(self)-> str:
        '''
        Fetching Data from url
        '''
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs('artifacts/data_ingestion', exist_ok= True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")
            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id,zip_download_dir)
            logger.info(f"Downloaded successful into file {zip_download_dir}")

        except Exception as e:
            raise e 
    
    def extract_zip_file(self):
        '''
        zip_file_path:str
        Extract the zip file into the data directory
        reture None
        '''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config
    data_ingestion = DataIngestion(config = data_ingestion_config())
    print(data_ingestion.config.source_URL)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()


except Exception as e:
    raise e

[2026-01-16 14:42:20,252: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-16 14:42:20,267: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-16 14:42:20,270: INFO: common: created directory at: artifacts]
[2026-01-16 14:42:20,271: INFO: common: created directory at: artifacts/data_ingestion]
https://drive.google.com/file/d/198Bae4WiV1faAEMtCQ8FOGRGx-1E0EEm/view?usp=sharing
[2026-01-16 14:42:20,273: INFO: 3048084552: Downloading data from https://drive.google.com/file/d/198Bae4WiV1faAEMtCQ8FOGRGx-1E0EEm/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=198Bae4WiV1faAEMtCQ8FOGRGx-1E0EEm
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=198Bae4WiV1faAEMtCQ8FOGRGx-1E0EEm&confirm=t&uuid=9fe8f06b-82b3-49ca-87f8-1af023b28dd0
To: c:\Users\Admin\Desktop\code\python\Mlops-Production-Ready-Deep-Learning-Project\artifacts\data_ingestion\data.zip
100%|██████████| 124M/124M [00:46<00:00, 2.67MB/s] 

[2026-01-16 14:43:10,791: INFO: 3048084552: Downloaded successful into file artifacts/data_ingestion/data.zip]
